# 보안, 금융, 위험 관리, 법률, 재해 복구, IT 거버넌스

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U langchain-community
!pip install transformers
!pip install datasets
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.3 MB/s eta 0:00:00


In [ ]:
import re
import json
import torch
import pandas as pd
import re
import tqdm

from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from langchain.document_loaders import PyPDFLoader
from statistics import mean
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

import warnings
warnings.filterwarnings("ignore")

# PDF Load & Split Chunk

In [ ]:
def pdf_load_split_chunk(file_path):
  loader = PyPDFLoader(file_path)
  documents = loader.load()

  # 불필요한 문자 제거
  pattern = r'[①-⑳◆·「」｢｣□◦\-\*❶-❸Ⅲ※‧Ÿ]'
  for doc in documents:
    text = doc.page_content
    text = re.sub(pattern, ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    doc.page_content = text

  paragraph_lengths = [len(doc.page_content) for doc in documents]

  print("문단 개수:", len(paragraph_lengths))
  print("평균 문단 길이:", round(mean(paragraph_lengths)))
  print("최소 문단 길이:", min(paragraph_lengths))
  print("최대 문단 길이:", max(paragraph_lengths))

  # 추천 chunk_size & overlap 계산
  avg_len = mean(paragraph_lengths)
  max_len = max(paragraph_lengths)

  # chunk_size는 최대 문단 길이보다 조금 크게, 평균의 2~3배 정도로 설정
  chunk_size = int(min(max_len * 1.1, avg_len * 3))
  chunk_overlap = int(chunk_size * 0.15)  # 15% 정도 겹치기

  print("추천 chunk_size:", chunk_size)
  print("추천 chunk_overlap:", chunk_overlap)

  text_spliter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
    separators = ["\n\n", "\n", " ", ""]
  )

  all_chunks_custom = text_spliter.split_documents(documents)
  print("청크의 수 :", len(all_chunks_custom))

  return all_chunks_custom

In [ ]:
# file_path1 = "/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/pdf/2025년도 금융감독개론.pdf"
file_path2 = '/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/pdf/금융분야 개인신용정보 보호 가이드라인(2025.5.) (1).pdf'

# pdf1_chunks = pdf_load_split_chunk(file_path1)
pdf2_chunks = pdf_load_split_chunk(file_path2)

문단 개수: 166
평균 문단 길이: 837
최소 문단 길이: 0
최대 문단 길이: 1588
추천 chunk_size: 1746
추천 chunk_overlap: 261
청크의 수 : 163


In [ ]:
# 처음 불필요한 부분 확인
# pdf1_chunks[5].page_content
pdf2_chunks[5].page_content

'4 Ⅱ. 기본원칙1 법 적용 원칙 개인정보보호법 은개인정보보호관련다른법률에특별한규정이있는경우를제외하고 개인정보보호법 에서정하는바를따르도록규정 개인정보 보호법 제6조 (다른 법률과의 관계) 개인정보의 처리 및 보호에 관하여 다른 법률에 특별한 규정이 있는 경우를 제외하고는 이 법에서 정하는 바에 따른다. 개인정보의 처리 및 보호에 관한 다른 법률을 제정하거나 개정하는 경우에는 이 법의 목적과 원칙에 맞도록 하여야 한다. 신용정보법 은신용정보의이용및보호와관련한원칙과기준을규정하고있으며, 개인정보보호법 에대한특별법임을명시 신용정보법 제3조의2 (다른 법률과의 관계) 신용정보의 이용 및 보호에 관하여 다른 법률에 특별한 규정이 있는 경우를 제외하고는 이 법에서 정하는 바에 따른다. 개인정보의 보호에 관하여 이 법에 특별한 규정이 있는 경우를 제외하고는 개인정보 보호법 에서 정하는 바에 따른다.2 주요 개념 정리 1) 개인정보의 개념 (개인정보 보호법 제2조 제1호) ‘개인정보’란살아있는개인에관한정보로서다음의어느하나에해당하는정보를말함 성명,주민등록번호및영상등을통하여개인을알아볼수있는정보 해당정보만으로는특정개인을알아볼수없더라도다른정보와쉽게결합하여알아볼수있는정보 이 경우 쉽게 결합할 수 있는지 여부는 다른 정보의 입수 가능성 등 개인을 알아보는 데 소요되는 시간, 비용, 기술 등을 합리적으로 고려하여야 함 개인정보를가명처리하여원래의상태로복원하기위한추가정보의사용 결합없이는특정개인을알아볼수없는정보(가명정보)'

In [ ]:
# pdf1_chunks = pdf1_chunks[5:]
# print("줄어든 청크 수 :", len(pdf1_chunks))
pdf2_chunks = pdf2_chunks[5:]
print("줄어든 청크 수 :", len(pdf2_chunks))

줄어든 청크 수 : 158


In [ ]:
# 마지막 불필요한 부분 확인
# pdf1_chunks[-20].page_content
pdf2_chunks[-2].page_content

'160 15 신용정보업Q15 1채권추심회사가 채권추심업무를 위하여 개인신용정보를 이용 또는 제공하거나 제공받은 경우 신용정보법 제35조에 따라 신용정보주체가 이를 조회할 수 있도록 하여야 하는지요?A신용정보주체에 대한 채권추심회사의 채권추심업무는 채권자로부터 위탁받은 업무이므로 채권추심회사가 위탁받은 업무 수행을 위하여 개인신용정보를 이용 또는 제공하거나 제공받은 경우는 신용정보법 제35조제1항 단서에 해당에 해당되므로 신용정보주체가 조회할 수 있도록 하지 않아도 됩니다.Q15 2채권추심회사가 채권추심의 대상이 되는 자의 개인신용정보를 채권자로부터 제공받거나 채권자에게 제공하는 경우, 신용정보회사로부터 제공받는 경우에, 제공하거나 제공받을 때마다 신용정보주체에게 통지하여야 하는지요? A 신용정보회사등은 개인신용정보를 제공하거나 제공받을 때마다 해당 신용정보주체로부터 개별적으로 동의를 받아야 하나, 기존에 동의한 목적 또는 이용 범위에서 개인신용정보의 정확성 최신성을 유지하기 위한 경우에는 동의를 면제하고 있습니다.( 신용정보법 제32조제1항 및 제2항) 신용정보법 제32조 제6항에서는 신용정보주체로부터 동의를 받지 않아도 되는 경우를 규정하고 있으며, 제32조 제7항에서는 제6항에 따라 동의를 받지 않는 경우에 신용정보주체에게 통지하도록 하고 있습니다.(채권추심은 제32조 제6항제4호에서 규정)따라서 신용정보법 제32조 제7항에 따른 통지의 경우에도, 제32조 제1항 및 제2항의 동의에 관한 규정에 따라, 정확성 최신성을 유지하기 위한 경우에는 다시 통지할 필요가 없습니다.Q15 3채권추심회사는 신용정보법 시행령 제28조제12항 및 별표 2의2 제4호부터 제6호에 따라 개인신용정보의 제공 사실 및 이유 등을 신용정보주체에게 알려야 하는데, 이러한 통지를 하나의 통지서(수임사실통지서 등)에 통합하여 발송해도 문제가 없는지?A채권추심회사가 통지를 하는 대상은 각각의 경우가 모두 채권추심의 대상이 되는 자로 동일하며, 채권자에게 채권추심의 대상이 되는 자의 개인신

In [ ]:
# 마지막 19페이지 제외
# pdf1_chunks = pdf1_chunks[:-20]
# print("줄어든 청크 수 :", len(pdf1_chunks))
pdf2_chunks = pdf2_chunks[:-2]
print("줄어든 청크 수 :", len(pdf2_chunks))

줄어든 청크 수 : 156


In [ ]:
# all_chunks = []

all_chunks.extend(pdf2_chunks)
print(len(all_chunks))

851


In [ ]:
# all_chunks 저장
serializable_chunks = []
for chunk in all_chunks:
    serializable_chunks.append({
        "page_content": chunk.page_content,
        "metadata": chunk.metadata
    })

with open("/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/all_chunks.json", "w", encoding = "utf-8") as f:
        json.dump(serializable_chunks, f, ensure_ascii = False, indent=4)